## Looking back at our first example

In [1]:
import math
import numpy as np
import tensorflow as tf
import keras

In [2]:
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

### A simple Dense class

```python
output = activation(matmul(input, W) + b)
```

In [3]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation=None):
        self.activation = activation
        self.W = keras.Variable(
            # Creates a matrix W of shape (input_size, output_size),
            # initialized with random values drawn from a uniform
            # distribution
            shape=(input_size, output_size), initializer="uniform"
        )
        # Creates a vector b of shape (output_size,), initialized with zeros
        self.b = keras.Variable(shape=(output_size,), initializer="zeros")

    # Applies the forward pass
    def __call__(self, inputs):
        # Store input for backpropagation
        self._stored_input = inputs
        x = keras.ops.matmul(inputs, self.W)
        x = x + self.b
        # Store pre-activation value for backpropagation
        self._stored_pre_activation = x
        if self.activation is not None:
            x = self.activation(x)
        return x

    @property
    # The convenience method for retrieving the layer's weights
    def weights(self):
        return [self.W, self.b]

### A simple Sequential class

In [4]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights


In [22]:
model = NaiveSequential(
    [
        NaiveDense(input_size=28 * 28, output_size=512, activation=keras.ops.relu),
        NaiveDense(input_size=512, output_size=10, activation=keras.ops.softmax),
    ]
)
assert len(model.weights) == 4

### A batch generator

In [6]:
class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

### Manual gradient computation (without GradientTape)

#### Gradient derivation

For the last layer with softmax activation and sparse categorical cross-entropy loss, we can derive a simple formula for the gradient.

##### The loss function: Cross-entropy & Gradient

For a single sample with true label $y$ (an integer) and predicted probabilities $\mathbf{p} = [p_1, p_2, ..., p_C]$ (where $C$ is the number of classes), the cross-entropy loss is:

$$L = -\log(p_y)$$

We want to compute $\frac{\partial L}{\partial p_i}$ for all $i$.

$$\frac{\partial L}{\partial p_y} = \frac{ \partial (-\log(p_y)) } { \partial p_y} = {\color{blue} -\frac{1}{p_y} } $$

##### The softmax function & Gradient

Given pre-softmax logits $\mathbf{z} = [z_1, z_2, ..., z_C]$, the softmax probabilities are:

$$p_i = \frac{e^{z_i} }{\sum_{j=1}^{C} e^{z_j} }$$

The derivative of softmax is:
$$\begin{align}
\frac{\partial p_i}{\partial z_y} & = \frac{\partial  \frac{ e^{z_i} \quad (= g(x) ) }{\sum_{j=1}^{C} e^{z_j}  \quad ( = h(x) )} }{\partial z_y} & | \quad \small \left( \frac{ g(x) }{ h(x) } \right)' = \frac{ {\color{pink} g'(x) } {\color{cyan} h(x) }- { \color{purple} h'(x) } { \color{orange} g(x) } }{ {\color{magenta} h(x)^2 } } \\
\\
& =
\begin{cases}
  \begin{aligned}
    & = \frac{ {\color{pink} e^{z_y} } {\color{cyan}  \sum_{j=1}^{C} e^{z_j} } -  { \color{purple} e^{z_y} }  { \color{orange}  e^{z_i} } }{  { \color{magenta} { \sum_{j=1}^{C} e^{z_j} }^2 } } \\
    & = \frac{ e^{z_y} }{  \sum_{j=1}^{C} e^{z_j} } \frac{ ( \sum_{j=1}^{C} e^{z_j} -  e^{z_i} ) }{ \sum_{j=1}^{C} e^{z_j} } \\
    \\
    & = \underline{ { \color{green} p_y(1 - p_i) }  }
  \end{aligned}
  & \text{if } i = y,\\
  \\
  \\
  \begin{aligned}
  & = \frac{ \overbrace{ 0 }^{ { \color{pink} 0 } \cdot {\color{cyan}  \sum_{j=1}^{C} e^{z_j} } } - { \color{purple} e^{z_y} }   { \color{orange} e^{z_i} } }{  { \color{magenta} { \sum_{j=1}^{C} e^{z_j} }^2 } } \\
  & = \frac{-  e^{z_y} }{  \sum_{j=1}^{C} e^{z_j} }  \frac{ e^{z_i} }{ \sum_{j=1}^{C} e^{z_j} } \\  
  \\
  & = \underline{ { \color{green} -p_y \cdot p_i } } 
  & \text{otherwise.}
  \end{aligned}
\end{cases} 
\end{align}$$

See [this page](https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/).

**Case 1**: $i = y$ (the true class):
$$
\begin{align}
\frac{\partial L}{\partial z_y} & = { \color{blue} -\frac{1}{p_y} } \cdot \frac{\partial p_y}{\partial z_y}\\
& = { \color{blue} -\frac{1}{p_y} }  \cdot { \color{green} p_y(1 - p_i) } \\
& = -(1 - p_i) \\
& = \underline{ { \color{red} p_i - 1 } }
\end{align}
$$

**Case 2**: $i \neq y$ (all others):
$$
\begin{aligned}
\frac{\partial L}{\partial z_i} & = { \color{blue} -\frac{1}{p_y} }  \cdot \frac{\partial p_y}{\partial z_i} \\
& =  { \color{blue} -\frac{1}{p_y} }  \cdot { \color{green} (-p_y \cdot p_i) }  \\
& = \underline{ { \color{red} p_i } }
\end{aligned}
$$

**Combining both cases:**
We can write this compactly using one-hot encoding. If $\mathbf{y}_{one-hot}$ is the one-hot vector of the true label, then:

$$\frac{\partial L}{\partial \mathbf{z}} = { \color{red} \mathbf{p} - \mathbf{y}_{one-hot} } $$

##### Average loss (mini-batch)

When we have a batch of $N$ samples, we compute the **average loss** over the batch:

$$L = \frac{1}{N} \sum_{i=1}^{N} L_i$$

To understand why we divide by $N$ in the gradient, consider the derivative of an average function. If $f(x_1, x_2, ..., x_N) = \frac{1}{N} \sum_{i=1}^{N} x_i$, then:

$$\frac{\partial f}{\partial x_i} = \frac{1}{N}$$

This is because when taking the partial derivative with respect to $x_i$, all other terms in the sum vanish (their derivatives are zero), we take the partial derivative with respect to the remaining term $\frac{1}{N} x_i$ and only the constant $\frac{1}{N}$ remains.

Applying this to our loss: since $L = \frac{1}{N} \sum_{i=1}^{N} L_i$, we have:

$$\frac{\partial L}{\partial \mathbf{z}_i} = \frac{1}{N} \cdot \frac{\partial L_i}{\partial \mathbf{z}_i} = \frac{1}{N}({ \color{red} \mathbf{p}_i - \mathbf{y}_{i,one-hot} })$$

For the entire batch, in vectorised form, this gives us:

$$\frac{\partial L}{\partial \mathbf{z}} = \frac{1}{N}({ \color{red} \mathbf{p} - \mathbf{y}_{one-hot} })$$

### Manual Implementation

Caveat: this is not how this is usually implemented. The current function is very restrictive, and would break as soon as we changed the loss or activation functions, or if we had any other change in our model graph. The principled way is to build classes for all operations (addition, multiplication, activation functions, etc.), that store what their imputs are whenever they are used, and automatically compute the backward pass (gradient). Each operation only needs to know about its own inputs and the incoming gradient(s) from previous nodes in the graph. That allows for a full automation and the computation of gradients on arbitary graphs. See the [reference, "Backpropagation", the Andrej Karpathy & Sasha Rush courses](https://github.com/jchwenger/DLWP/tree/main/lectures/02/references.ipynb) for more details.

In [7]:
def get_gradients_of_loss_wrt_weights(model, images_batch, labels_batch, predictions, loss):
    """
    Computes gradients of loss w.r.t. weights using manual backpropagation.
    
    This function implements backpropagation from scratch without using GradientTape.
    It works with NaiveSequential models containing NaiveDense layers.
    """
    batch_size = keras.ops.shape(labels_batch)[0]
    num_classes = keras.ops.shape(predictions)[1]
    
    gradients = []
    
    # Backpropagate through each layer in reverse order    
    for layer_idx in range(len(model.layers) - 1, -1, -1):
        layer = model.layers[layer_idx]
        layer_input = layer._stored_input
        pre_activation = layer._stored_pre_activation
        
        # Backpropagate through activation function
        if layer_idx == len(model.layers) - 1:
            # For softmax + sparse_categorical_crossentropy, the gradient w.r.t. pre-softmax logits
            # simplifies to: (predictions - one_hot_labels) / batch_size
            labels_one_hot = keras.ops.one_hot(labels_batch, num_classes)
            current_grad = (predictions - labels_one_hot) / keras.ops.cast(batch_size, 'float32')
        else:
            # ReLU gradient: 1 if x > 0, else 0
            relu_mask = keras.ops.cast(pre_activation > 0, 'float32')
            current_grad = current_grad * relu_mask
 
        # Compute gradients w.r.t. weights and bias
        grad_W = keras.ops.matmul(keras.ops.transpose(layer_input), current_grad)
        grad_b = keras.ops.sum(current_grad, axis=0)
        
        # Store gradients (in reverse order)
        gradients.insert(0, grad_b)
        gradients.insert(0, grad_W)
        
        # Compute gradient w.r.t. input for previous layer
        # (in the forward pass, the output of the activation is now x, multiplied by w)
        if layer_idx > 0:
            current_grad = keras.ops.matmul(current_grad, keras.ops.transpose(layer.W))
    
    return gradients

### Sanity checks

In [8]:
with tf.GradientTape() as tape:
    predictions = model(train_images[:5])
    loss = keras.ops.sparse_categorical_crossentropy(train_labels[:5], predictions)
    average_loss = keras.ops.mean(loss)

In [9]:
gradients_manual = get_gradients_of_loss_wrt_weights(model, train_images[:5], train_labels[:5], predictions, average_loss)

In [10]:
gradients_tf = tape.gradient(average_loss, model.weights)

In [11]:
# Compare gradients with tolerance for floating-point precision
for i, (grad_man, grad_tf) in enumerate(zip(gradients_manual, gradients_tf)):
    is_close = np.allclose(grad_man.numpy(), grad_tf.numpy())
    max_diff = keras.ops.max(keras.ops.abs(grad_man - grad_tf))
    print(f"Gradient {i}: allclose={is_close}, max_diff={max_diff:.2e}")

Gradient 0: allclose=True, max_diff=4.19e-09
Gradient 1: allclose=True, max_diff=4.19e-09
Gradient 2: allclose=True, max_diff=1.49e-08
Gradient 3: allclose=True, max_diff=1.49e-08


### Gradient computation in practice

`tf.GradientTape` automatically computes the whole function above.

In [12]:
def one_training_step_keras(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        loss = keras.ops.sparse_categorical_crossentropy(labels_batch, predictions)
        average_loss = keras.ops.mean(loss)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

## Running one training step

### The weight update step

In [13]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        # Assigns a new value to the variable, in place
        w.assign(w - g * learning_rate)

In practice, you'd use the implemented class (an object-oriented approach makes implementing running averages fairly neat, as the class object can hold these).

In [14]:
optimizer = keras.optimizers.SGD(learning_rate=1e-3)

def update_weights_keras(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [15]:
def one_training_step(model, images_batch, labels_batch):
    # Runs the "forward pass"
    predictions = model(images_batch)
    loss = keras.ops.sparse_categorical_crossentropy(labels_batch, predictions)
    average_loss = keras.ops.mean(loss)
    # Computes the gradient of the loss with regard to the weights. The
    # output, gradients, is a list where each entry corresponds to a
    # weight from the model.weights list.
    # Pass the per-sample loss, not the averaged one
    gradients = get_gradients_of_loss_wrt_weights(model, images_batch, labels_batch, predictions, loss)
    # Updates the weights using the gradients.
    update_weights(gradients, model.weights)
    return average_loss

## The full training loop

In [20]:
def fit(model, images, labels, epochs, batch_size=128):
    # loop over the entire dataset for a number of epochs
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter+1:>02}")
        # instantiate our batch generator
        batch_generator = BatchGenerator(images, labels)
        # loop over all mini-batches in the dataset
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter:>03}: {loss:.2f}")
        print()

In [23]:
fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 01
loss at batch 000: 2.31
loss at batch 100: 2.28
loss at batch 200: 2.24
loss at batch 300: 2.20
loss at batch 400: 2.15

Epoch 02
loss at batch 000: 2.13
loss at batch 100: 2.12
loss at batch 200: 2.07
loss at batch 300: 2.03
loss at batch 400: 1.98

Epoch 03
loss at batch 000: 1.95
loss at batch 100: 1.96
loss at batch 200: 1.88
loss at batch 300: 1.85
loss at batch 400: 1.81

Epoch 04
loss at batch 000: 1.75
loss at batch 100: 1.78
loss at batch 200: 1.68
loss at batch 300: 1.66
loss at batch 400: 1.63

Epoch 05
loss at batch 000: 1.55
loss at batch 100: 1.59
loss at batch 200: 1.48
loss at batch 300: 1.47
loss at batch 400: 1.46

Epoch 06
loss at batch 000: 1.36
loss at batch 100: 1.42
loss at batch 200: 1.29
loss at batch 300: 1.30
loss at batch 400: 1.30

Epoch 07
loss at batch 000: 1.20
loss at batch 100: 1.26
loss at batch 200: 1.13
loss at batch 300: 1.15
loss at batch 400: 1.17

Epoch 08
loss at batch 000: 1.06
loss at batch 100: 1.12
loss at batch 200: 0.99
loss at b

## Evaluating the model

In [24]:
predictions = model(test_images)
predicted_labels = keras.ops.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
f"accuracy: {keras.ops.mean(matches):.2f}"

'accuracy: 0.84'